In [73]:
import numpy as np
import pandas as pd
p=print

## Is Your Girlfriend Cheating on You?

Your girlfriend Mary has been exhibiting odd sequences of behavior. You suspect cheating. You don't want to falsely accuse her but something is off. Is there a way to reason about the situation so you can decide if you should confront her using the limited information you have?

You recall Hidden Markov Models are a tool for these situations. Mary's activities represent a sequence of hidden states, with the observed behavior a sequence of emissions. Because you don't talk much and you believe she may lie to you, all you can do is try to guess her true state via observations taken over time. 

First you define the range of possible states **`M`**. You know Mary has a strong work ethic, both professionally and actively. So you conclude that when not with you, she could be at __Work, Gym, or Cheating__. 

These states are hidden to you and cannot be observed directly.

You do not know the initial probabilities **`pi`**, of which state she could be in. You decide that Work or the Gym is equiprobable, and that there is a small percent she is cheating.

<center>**pi = [0.4, 0.4, 0.2]**</center>

In [74]:
# define states
# work --> gym --> cheating

# initial probability of being in state k, for M states
states = ['work', 'gym', 'cheating']
pi = np.array([0.4, 0.4, 0.2])

# pi df
(pd.Series(pi, index=states, name='states'))

work        0.4
gym         0.4
cheating    0.2
Name: states, dtype: float64

You then have to guess about the transition probabilities for the matrix of possible states. If Mary was working what is the probability she would continue working, then transition to the gym, then transition to cheating? 

These are difficult questions to ask, no doubt, but you push forward.

You start with the state transitions for work. You reason that the probability is high, that Mary would keep working given she is already working. It is also highly probable she could transition from work to the gym. You assign a low probability that she could transition from work to cheating.

<center>**work = [0.6, 0.3, 0.1]**</center>

The gym is less certain. You reason she could transition from the gym state to any other state with equal probability.

<center>**gym = [0.33, 0.33, 0.33]**</center>

Finally, you consider if she were cheating, that you have no idea what state she would transition to afterwards. 

<center>**cheating = [0.33, 0.33, 0.33]**</center>

In [75]:
# a or alpha = transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states

a_df = pd.DataFrame(columns=states, index=states)
a_df.loc[states[0]] = [.6, .3, .1]
a_df.loc[states[1]] = [.33, .33, .33]
a_df.loc[states[2]] = [.33, .33, .33]
p(a_df)

a = a_df.values
p('\n',a, a.shape)

          work   gym cheating
work       0.6   0.3      0.1
gym       0.33  0.33     0.33
cheating  0.33  0.33     0.33

 [[0.6 0.3 0.1]
 [0.33 0.33 0.33]
 [0.33 0.33 0.33]] (3, 3)


The final requirement is to reason about the observation aka _emission_ probabilities. These are the probabilities that you would observe a particular behavior given she is in a particular state.

Again you do not know Mary's true states because you don't talk, and you believe she may lie to you. Instead you focus on observations you believe are linked to her true state. 

These observations are __makeup, athletic dress, and locked cell phone.__ 

Given Mary is in the work state, it is highly probable that she would wear makeup to work, very low probability that she would dress athletically, and high probability she would lock her phone.

<center>__work_emission = [0.4, 0.1, 0.5]__</center>

Mary is an avid gym goer. Given the gym state, she is unlikely to wear makeup, likely to dress athletically, and is very likely to lock her phone.

<center>__gym_emission = [0.1, 0.3, 0.6]__</center>

If she is cheating, you figure you clearly don't know Mary like you thought, and you certainly do not know the probability that she will emit any of these behaviors if she is cheating therefore you set them equiprobable. 

<center> __cheating_emission = [0.33, 0.33, 0.33]__ </center>

In [76]:
# Emission probabilities
# b or beta = observation probabilities given state
# matrix is size (M x O) where M is number of states 
# and O is number of different possible observations

emit = ['makeup', 'dress', 'phone']
b_df = pd.DataFrame(columns=emit, index=states)
b_df.loc[states[0]] = [0.4, 0.1, 0.5]
b_df.loc[states[1]] = [0.1, 0.3, 0.6]
b_df.loc[states[2]] = [0.33, 0.33, 0.33]
p(b_df)

b = b_df.values
p('\n', b, b.shape)

         makeup dress phone
work        0.4   0.1   0.5
gym         0.1   0.3   0.6
cheating   0.33  0.33  0.33

 [[0.4 0.1 0.5]
 [0.1 0.3 0.6]
 [0.33 0.33 0.33]] (3, 3)


Now we simply record the observation sequence

In [77]:
# observation sequence of Mary's behaviors
# observations are encoded numerically

obs_map = {'makeup':0, 'dress':1, 'phone':2}
obs = np.array([0,2,2,1,1,0,2,1,2,1,1,2,0,2])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

p( pd.DataFrame(np.column_stack([obs, obs_seq]), 
                columns=['Obs_code', 'Obs_seq']) )

   Obs_code Obs_seq
0         0  makeup
1         2   phone
2         2   phone
3         1   dress
4         1   dress
5         0  makeup
6         2   phone
7         1   dress
8         2   phone
9         1   dress
10        1   dress
11        2   phone
12        0  makeup
13        2   phone


## The HMM can answer the question, _given this sequence of observed behaviors and our model parameters, what is the most likely sequence of hidden states?_

You can calculate this using the __Viterbi__ algorithm.

High level, the Viterbi algorithm increments over each time step, finding the _maximum_ probability of any path that gets to state `i` at time `t`, that _also_ has the correct observations for the sequence up to time `t`.

The algorithm also keeps track of the state with the highest probability at each stage. At the end of the sequence, the algorithm will iterate backwards selecting the state that "won" each time step, and thus creating the most likely path, or likely sequence of hidden states that led to the sequence of observations. 

In [78]:
# define Viterbi algorithm for shortest path
# taken from Stephen Marsland's, Machine Learning An Algorthmic Perspective, Vol. 2
# https://github.com/alexsosn/MarslandMLAlgo/blob/master/Ch16/HMM.py

def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = np.zeros(T)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    p('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            p('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    p('-'*50)
    p('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]
        p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
        p('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

path, delta, phi = viterbi(pi, a, b, obs)
p('\nsingle best state path: \n', path)
p('delta:\n', delta)
p('phi:\n', phi)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 0.0
s=1 and t=1: phi[1, 1] = 0.0
s=2 and t=1: phi[2, 1] = 2.0
s=0 and t=2: phi[0, 2] = 0.0
s=1 and t=2: phi[1, 2] = 0.0
s=2 and t=2: phi[2, 2] = 1.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 0.0
s=2 and t=3: phi[2, 3] = 1.0
s=0 and t=4: phi[0, 4] = 0.0
s=1 and t=4: phi[1, 4] = 1.0
s=2 and t=4: phi[2, 4] = 1.0
s=0 and t=5: phi[0, 5] = 2.0
s=1 and t=5: phi[1, 5] = 2.0
s=2 and t=5: phi[2, 5] = 2.0
s=0 and t=6: phi[0, 6] = 0.0
s=1 and t=6: phi[1, 6] = 0.0
s=2 and t=6: phi[2, 6] = 2.0
s=0 and t=7: phi[0, 7] = 0.0
s=1 and t=7: phi[1, 7] = 0.0
s=2 and t=7: phi[2, 7] = 1.0
s=0 and t=8: phi[0, 8] = 0.0
s=1 and t=8: phi[1, 8] = 1.0
s=2 and t=8: phi[2, 8] = 1.0
s=0 and t=9: phi[0, 9] = 0.0
s=1 and t=9: phi[1, 9] = 1.0
s=2 and t=9: phi[2, 9] = 1.0
s=0 and t=10: phi[0, 10] = 0.0
s=1 and t=10: phi[1, 10] = 2.0
s=2 and t=10: phi[2, 10] = 2.0
s=0 and t=11: phi[0, 11] = 2.0
s=1 and t=11: phi[1, 11] = 2.0
s=2 and t=11: phi[2, 11] = 2.0
s=0 and t=

/home/bcr/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [79]:
(pd.DataFrame(delta, index=states).T)

,work,gym,cheating
0,1.600000e-01,4.000000e-02,6.600000e-02
1,4.800000e-02,2.880000e-02,7.187400e-03
2,1.440000e-02,8.640000e-03,3.136320e-03
3,8.640000e-04,1.296000e-03,9.408960e-04
4,5.184000e-05,1.283040e-04,1.411344e-04
5,1.862974e-05,4.657435e-06,1.536954e-05
6,5.588922e-06,3.353353e-06,1.673742e-06
7,3.353353e-07,5.030030e-07,3.651802e-07
8,1.006006e-07,9.959459e-08,5.477703e-08
9,6.036036e-09,9.859865e-09,1.084585e-08


In [80]:
state_map = {0:'work', 1:'gym', 2:'cheating'}
state_path = [state_map[v] for v in path]

(pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))

,Observation,Best_Path
0,makeup,work
1,phone,work
2,phone,work
3,dress,gym
4,dress,cheating
5,makeup,work
6,phone,work
7,dress,gym
8,phone,gym
9,dress,cheating


In [81]:
#from IPython.core.display import HTML
#HTML(open("./Hidden_Markov_Models/custom.css", "r").read())


# Modified Stylesheet for notebook.
from IPython.core.display import HTML
import requests

def css_styling():
    css = requests.get('https://github.com/BlackArbsCEO/Hidden_Markov_Models/blob/master/custom.css')
    return HTML("<style>"+css.content.decode('UTF-8')+"</style>")

css_styling()